In [64]:
!rm -r labor-market-analysis
!git clone https://github.com/mia-uc/labor-market-analysis.git
%cd labor-market-analysis

# !pip install -r requirements.txt
!pip install openai pandas

In [ ]:
import pandas as pd
import openai


In [68]:
urls = [
'./asserts/githut-csv/gh-issue-event.json.csv',
'./asserts/githut-csv/gh-pull-request.json.csv',
'./asserts/githut-csv/gh-push-event.json.csv',
'./asserts/githut-csv/gh-star-event.json.csv',
'./asserts/archive/prs.csv',
'./asserts/archive/issues.csv',
]

final_dp = pd.read_csv('./asserts/archive/repos.csv')
final_dp['name'] = final_dp['language']

for url in urls:
    df = pd.read_csv(url)
    final_dp = pd.concat([final_dp, df['name']])

final_dp['name'] = final_dp[0].str.lower()
pls = final_dp[['name']].drop_duplicates().sort_values(by='name')
pls = pls[pls['name'].isnull() == False]
pls.set_index('name')

print(len(pls))

322


In [69]:
pls['name'].unique()


array(['1c enterprise', 'abap', 'actionscript', 'ada',
       'adblock filter list', 'agda', 'ags script', 'alloy', 'ampl',
       'antlr', 'apacheconf', 'apex', 'api blueprint', 'apl',
       'applescript', 'arc', 'arduino', 'asp', 'assembly', 'astro', 'ats',
       'augeas', 'autohotkey', 'autoit', 'awk', 'batchfile', 'bicep',
       'bison', 'bitbake', 'blade', 'blitzbasic', 'bluespec', 'boo',
       'brainfuck', 'brightscript', 'bro', 'c', 'c#', 'c++', 'cartocss',
       'ceylon', 'chapel', 'clean', 'click', 'clojure', 'cmake', 'cobol',
       'codeql', 'coffeescript', 'coldfusion', 'common lisp',
       'common workflow language', 'component pascal', 'coq', 'crystal',
       'css', 'cucumber', 'cuda', 'cweb', 'cython', 'd', 'dart',
       'dcpu-16 asm', 'delphi', 'diff', 'digital command language', 'dm',
       'dockerfile', 'dot', 'dtrace', 'dylan', 'eagle', 'ecl', 'eiffel',
       'ejs', 'elixir', 'elm', 'emacs lisp', 'emberscript', 'erlang',
       'f#', 'f*', 'factor', 'fancy'

In [71]:
SYSTEM_CONFIG = """\
The user have a list of name which are supposedly a programming language's name. \
So, you are a bot to classifier them in programming language's name or wrong name. \
The user will give you a name and you should answer if this is a programming language's name or not. \
If the name is a programming language's name you have to answer with some information about it. \
As example:

user: Isabelle?
assistant: Isabelle is a programming language. \
It was developed in the 1980s by German philosopher and computer scientist Tobias Nipkow and British computer scientist Lawrence Paulson.

Isabelle is a general-purpose programming language used primarily for formal verification of software and hardware systems. \
It is also used in areas of research in artificial intelligence, natural language processing, and hardware verification.

Isabelle is considered a predicate calculus-based language, which allows users to express precise, \
formal logical statements that can be tested automatically using a variety of methods and techniques.

user: Daniel?
assistant: I don't know any programming language which names Daniel.
"""

conversation = [{"role": 'system', 'content': SYSTEM_CONFIG}]

result = []
for i, language in enumerate(pls['name']):
    conversation.append({
        "role": 'user',
        "content": f"{language}?"
    })

    response = openai.ChatCompletion.create(
        model = 'gpt-3.5-turbo',
        messages = conversation
    )
    
    conversation.append({
        "role": 'assistant',
        "content": response.choices[0].message.content.strip()
    })

    result.append({
        'name' : language,
        'description': response.choices[0].message.content.strip()
    })

0


KeyboardInterrupt: 

In [60]:
result = pd.DataFrame(result)
result.to_csv('programming_languages.csv', index=False)